# Tuning Long Short-Term Memory Recurrent Neural Network

In [2]:
import numpy
import csv
import time
import pandas
import signal
import warnings
import random
import pickle
import tensorflow as tf
from tensorflow import set_random_seed
set_random_seed(159)
numpy.random.seed(159)
random.seed(159)
tf.logging.set_verbosity(tf.logging.ERROR)
warnings.filterwarnings("ignore")

from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras.metrics import categorical_accuracy

Using TensorFlow backend.


In [3]:
cmb = pandas.read_csv('05 Combined Cleaned.csv', delimiter=',', encoding='latin-1').fillna('')
cmb[:1]

,ID,Process Step,Problem Type,Contributing Factors,Overall Severity,Incident Description,Language,Translated,Neat Cleaned,Bony Cleaned
0,2511,Treatment delivery,"Wrong, missing, mislabeled, or damaged treatme...",Distraction or diversions involving staff,None,Non prescribed bolus . Bolus was not prescribe...,En,Non prescribed bolus . Bolus was not prescribe...,non prescribed bolus . bolus was not prescribe...,non prescribe bolus bolus prescribe md pt rece...


## Problem Type

In [4]:
pt_ohe = pandas.read_csv('07 NM PT OHE.csv', delimiter=',', encoding='latin-1').fillna('')
pt_ohe[:1]

,Wrong target or OAR contours,Treatment plan acceptable but not physically deliverable,Excess imaging dose,Failure to perform on-treatment imaging as per instructions,Wrong patient,Fall or other patient injury or medical condition,Wrong anatomical site (excluding laterality),Treatment plan (isodose distribution) unacceptable,Systematic hardware/software (including dose-volume) error,Interventional procedure error (Retired value),...,Inappropriate or poorly informed decision to treat or plan,Inadequate coordination of combined modality care,"Wrong, missing, mislabeled, or damaged treatment accessories",Wrong side (laterality),Wrong prescription dose-fractionation or calculation error,Radiation therapy scheduling error,Treatment not delivered - personnel/hardware/software failure,"Wrong patient position, setup point, or shift",Wrong planning margins,Allergic reaction
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [5]:
def lowest_correct(trues, preds):
    num_of_options = len(trues)
    drop_down_options = list(reversed(numpy.argsort(preds)))
    correct_options = [i for i in range(num_of_options) if trues[i]==1]
    return max([drop_down_options.index(correct_option) for correct_option in correct_options]) + 1
def average_lowest_correct(list_of_trues, list_of_preds):
    length = len(list(list_of_trues))
    return numpy.mean([lowest_correct(list(list_of_trues.iloc[i]), list(list_of_preds[i])) for i in range(length)])

In [6]:
nm_pt = cmb[cmb['Problem Type']!='']
txt = nm_pt['Bony Cleaned']
tkn = Tokenizer()
tkn.fit_on_texts(txt)
vcb_sz = len(tkn.word_index) + 1
encoded = tkn.texts_to_sequences(txt)

In [7]:
# embeddings_index = dict()
# f = open('glove.6B.100d.txt')
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = numpy.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

In [8]:
# embedding_matrix = numpy.zeros((vcb_sz, 100))
# for index, word in enumerate(tkn.word_index):
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[index] = embedding_vector
#     else:
#         embedding_matrix[index] = [random.uniform(-4, 4) for i in range(100)]

In [9]:
# with open('27 Embedding Matrix', 'wb') as em:
#     pickle.dump(embedding_matrix, em)

In [10]:
with open ('27 Embedding Matrix', 'rb') as em:
    embedding_matrix = pickle.load(em)

In [11]:
def deadline(timeout, *args):
    def decorate(f):
        def handler(signum, frame):
            raise Exception

        def new_f(*args):
            signal.signal(signal.SIGALRM, handler)
            signal.alarm(timeout)
            return f(*args)
            signal.alarm(0)

        new_f.__name__ = f.__name__
        return new_f
    return decorate

In [12]:
@deadline(600)
def lstm_cv(k, max_len, lstm_num, layers, lss, optimize, epoc, batch):
    
    kf = KFold(n_splits=k, shuffle=True)

    scrs = []

    for trn_inds, tst_inds in kf.split(encoded):

        trn_xs = [encoded[i] for i in trn_inds]
        trn_ys = pt_ohe.iloc[list(trn_inds)]

        tst_xs = [encoded[i] for i in tst_inds]
        tst_ys = pt_ohe.iloc[list(tst_inds)]

        trn_xs = sequence.pad_sequences(trn_xs, maxlen=max_len)
        tst_xs = sequence.pad_sequences(tst_xs, maxlen=max_len)
        
        model = Sequential()
        model.add(Embedding(vcb_sz, 100, input_length=max_len, weights=[embedding_matrix], trainable=True))
        model.add(LSTM(lstm_num))
        for layer, activate in layers:
            model.add(Dense(layer, activation=activate))
        model.compile(loss=lss, optimizer=optimize, metrics=['categorical_accuracy'])

        history = model.fit(trn_xs, trn_ys, epochs=epoc, batch_size=batch, verbose=0, validation_data=(tst_xs, tst_ys))
        scrs.append(average_lowest_correct(tst_ys, model.predict(tst_xs)))

    return numpy.mean(scrs).round(2)

In [13]:
act_grid = ['softmax', 'elu', 'selu', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
lss_grid = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error', 'squared_hinge', 'hinge', 'categorical_hinge', 'logcosh']
opt_grid = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Nadam', 'TFOptimizer']

In [14]:
# Let's grid search over 5 epochs as accuracy does not increase thereafter.

# Run all cells from above

In [14]:
# lstm_cv(3, 100, 100, [(100, 'relu'), (24, 'softmax')], 'mean_squared_error', 'Adagrad', 5, 10)
# 4.24

In [15]:
# lstm_cv(3, 100, 100, [(100, 'relu'), (24, 'softmax')], 'mean_squared_error', 'Adagrad', 5, 10)
# 4.28

In [16]:
# lstm_cv(3, 100, 100, [(100, 'relu'), (24, 'softmax')], 'mean_squared_error', 'Adagrad', 5, 10)
# 4.67

Models used to give very varied results. After setting the initial embedding weights and the seeds of the random number generators, the results are more reproducible.

In [17]:
# for lss in lss_grid:
#     for opt in opt_grid:
#         try:
#             print(lss, opt, lstm_cv(3, 100, 100, [(100, 'relu'), (24, 'softmax')], lss, opt, 5, 10))
#         except:
#             pass

# mean_squared_error SGD 13.58
# mean_squared_error RMSprop 4.35
# mean_squared_error Adagrad 4.5
# mean_squared_error Adadelta 5.21
# mean_squared_error Adam 4.58
# mean_squared_error Nadam 4.82
# mean_absolute_error SGD 9.5
# mean_absolute_error RMSprop 8.29
# mean_absolute_error Adagrad 6.08
# mean_absolute_error Adadelta 6.6
# mean_absolute_error Adam 6.38
# mean_absolute_error Nadam 6.83
# mean_absolute_percentage_error SGD 13.43
# mean_absolute_percentage_error RMSprop 5.69
# mean_absolute_percentage_error Adagrad 7.54
# mean_absolute_percentage_error Adadelta 7.07
# mean_absolute_percentage_error Adam 4.46
# mean_absolute_percentage_error Nadam 6.12
# mean_squared_logarithmic_error SGD 12.54
# mean_squared_logarithmic_error RMSprop 4.5
# mean_squared_logarithmic_error Adagrad 4.92
# logcosh SGD 14.03
# logcosh RMSprop 4.75
# logcosh Adagrad 4.68
# logcosh Adadelta 5.72
# logcosh Adam 4.62
# logcosh Nadam 4.9

In [18]:
rsl = '''mean_squared_error SGD 13.58
mean_squared_error RMSprop 4.35
mean_squared_error Adagrad 4.5
mean_squared_error Adadelta 5.21
mean_squared_error Adam 4.58
mean_squared_error Nadam 4.82
mean_absolute_error SGD 9.5
mean_absolute_error RMSprop 8.29
mean_absolute_error Adagrad 6.08
mean_absolute_error Adadelta 6.6
mean_absolute_error Adam 6.38
mean_absolute_error Nadam 6.83
mean_absolute_percentage_error SGD 13.43
mean_absolute_percentage_error RMSprop 5.69
mean_absolute_percentage_error Adagrad 7.54
mean_absolute_percentage_error Adadelta 7.07
mean_absolute_percentage_error Adam 4.46
mean_absolute_percentage_error Nadam 6.12
mean_squared_logarithmic_error SGD 12.54
mean_squared_logarithmic_error RMSprop 4.5
mean_squared_logarithmic_error Adagrad 4.92
logcosh SGD 14.03
logcosh RMSprop 4.75
logcosh Adagrad 4.68
logcosh Adadelta 5.72
logcosh Adam 4.62
logcosh Nadam 4.9'''

In [19]:
lst = []
for ln in rsl.split('\n'):
    elms = ln.split()
    elms[2] = float(elms[2])
    lst.append(elms)
lss_opt = pandas.DataFrame(lst, columns=['Loss', 'Optimizer', 'Score']).sort_values('Score')
lss_opt.to_csv('27 Losses and Optimizers.csv', encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)
lss_opt

,Loss,Optimizer,Score
1,mean_squared_error,RMSprop,4.35
16,mean_absolute_percentage_error,Adam,4.46
2,mean_squared_error,Adagrad,4.50
19,mean_squared_logarithmic_error,RMSprop,4.50
4,mean_squared_error,Adam,4.58
25,logcosh,Adam,4.62
23,logcosh,Adagrad,4.68
22,logcosh,RMSprop,4.75
5,mean_squared_error,Nadam,4.82
26,logcosh,Nadam,4.90


In [20]:
# for act1 in act_grid[3:]:
#     for act2 in act_grid:
#         try:
#             print(act1, act2, lstm_cv(5, 100, 100, [(100, act1), (24, act2)], 'mean_squared_error', 'RMSprop', 10, 50))
#         except:
#             print('Skpped ' + act1 + ' ' + act2)

In [21]:
rsl = '''softmax softmax 4.61
softmax elu 3.94
softmax selu 4.31
softmax softplus 10.81
softmax softsign 4.37
softmax relu 5.26
softmax tanh 5.32
softmax sigmoid 9.7
softmax hard_sigmoid 9.01
softmax linear 4.62
elu softmax 3.72
elu elu 3.84
elu selu 4.94
elu softplus 4.13
elu softsign 4.26
elu relu 5.99
elu tanh 4.25
elu sigmoid 4.3
elu hard_sigmoid 5.78
selu softmax 4.31
selu elu 4.24
selu selu 5.28
selu softplus 4.14
selu softsign 4.88
selu relu 6.01
selu tanh 4.36
selu sigmoid 4.01
selu hard_sigmoid 5.88
selu linear 4.42
softplus softmax 3.79
softplus elu 5.27
softplus selu 5.44
softplus softplus 3.99
softplus softsign 4.3
softplus relu 10.5
softplus tanh 4.99
softplus sigmoid 3.93
softplus hard_sigmoid 5.24
softplus linear 4.97
softsign softmax 4.08
softsign elu 5.1
softsign selu 4.4
softsign softplus 3.9
softsign softsign 4.77
softsign relu 6.48
softsign tanh 4.88
softsign sigmoid 4.27
softsign hard_sigmoid 5.57
softsign linear 4.05
relu softmax 4.06
relu elu 4.27
relu selu 4.82
relu softplus 4.03
relu softsign 4.66
relu relu 6.88
relu tanh 4.73
relu sigmoid 4.51
relu hard_sigmoid 6.06
relu linear 5.11
tanh softmax 4.27
tanh elu 5.04
tanh selu 4.11
tanh softplus 3.99
tanh softsign 4.57
tanh relu 6.61
tanh tanh 4.62
tanh sigmoid 4.28
tanh hard_sigmoid 5.38
tanh linear 3.98
sigmoid softmax 4.22
sigmoid elu 5.35
sigmoid selu 4.17
sigmoid softplus 3.99
sigmoid softsign 5.29
sigmoid relu 9.97
sigmoid tanh 5.38
sigmoid sigmoid 4.18
sigmoid hard_sigmoid 5.81'''

In [23]:
lst = []
for ln in rsl.split('\n'):
    elms = ln.split()
    elms[2] = float(elms[2])
    lst.append(elms)
acts = pandas.DataFrame(lst, columns=['Hidden', 'Output', 'Score']).sort_values('Score')
acts.to_csv('27 Activations.csv', encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)
acts

,Hidden,Output,Score
10,elu,softmax,3.72
29,softplus,softmax,3.79
11,elu,elu,3.84
42,softsign,softplus,3.90
36,softplus,sigmoid,3.93
1,softmax,elu,3.94
68,tanh,linear,3.98
72,sigmoid,softplus,3.99
32,softplus,softplus,3.99
62,tanh,softplus,3.99


In [25]:
for i in range(10, 150, 10):
    try:
        print(i, lstm_cv(5, 100, 100, [(i, 'elu'), (24, 'softmax')], 'mean_squared_error', 'RMSprop', 10, 50))
    except:
        print('Skpped ' + str(i))

10 4.38
20 4.0
30 4.31
40 4.58
50 4.68
60 4.64
70 4.88
80 4.72
90 4.05
100 4.82
110 4.5
120 5.16
130 4.28
140 4.65


In [26]:
for i in range(10, 30, 1):
    try:
        print(i, lstm_cv(5, 100, 100, [(i, 'elu'), (24, 'softmax')], 'mean_squared_error', 'RMSprop', 10, 50))
    except:
        print('Skpped ' + str(i))

10 4.17
11 4.43
12 4.36
13 4.59
14 4.45
15 4.51
16 3.76
17 4.18
18 4.59
19 4.39
20 4.87
21 4.32
22 3.9
23 4.42
24 4.54
25 4.74
26 4.48
27 4.75
Skpped 28
29 4.62


In [27]:
for i in range(20, 100, 20):
    try:
        print(i, lstm_cv(5, 100, i, [(22, 'elu'), (24, 'softmax')], 'mean_squared_error', 'RMSprop', 10, 50))
    except:
        print('Skpped ' + str(i))

20 4.28
40 4.64
60 4.51
80 4.47


In [15]:
for i in range(100, 180, 20):
    try:
        print(i, lstm_cv(5, 100, i, [(22, 'elu'), (24, 'softmax')], 'mean_squared_error', 'RMSprop', 10, 50))
    except:
        print('Skpped ' + str(i))

100 4.57
120 4.47
140 4.56
160 4.11


# Past

In [16]:
rsl = '''mean_squared_error SGD 14.25
mean_squared_error RMSprop 6.17
mean_squared_error Adagrad 5.75
mean_squared_error Adadelta 6.29
mean_squared_error Adam 3.75
mean_squared_error Nadam 5.5
mean_absolute_error SGD 17.0
mean_absolute_error RMSprop 6.12
mean_absolute_error Adagrad 5.58
mean_absolute_error Adadelta 5.54
mean_absolute_error Adam 6.67
mean_absolute_error Nadam 5.92
mean_absolute_percentage_error SGD 10.96
mean_absolute_percentage_error RMSprop 6.71
mean_absolute_percentage_error Adagrad 5.38
mean_absolute_percentage_error Adadelta 7.08
mean_absolute_percentage_error Adam 8.17
mean_absolute_percentage_error Nadam 4.75
mean_squared_logarithmic_error SGD 18.17
mean_squared_logarithmic_error RMSprop 4.92
mean_squared_logarithmic_error Adagrad 5.12
mean_squared_logarithmic_error Adadelta 4.88
mean_squared_logarithmic_error Adam 3.5
mean_squared_logarithmic_error Nadam 5.0
squared_hinge SGD 17.83
squared_hinge RMSprop 5.33
squared_hinge Adagrad 7.33
squared_hinge Adadelta 5.96
squared_hinge Adam 5.5
squared_hinge Nadam 8.17
hinge SGD 14.25
hinge RMSprop 5.04
hinge Adagrad 6.08
hinge Adadelta 10.12
hinge Adam 4.92
hinge Nadam 6.75
categorical_hinge SGD 13.62
categorical_hinge RMSprop 6.71
categorical_hinge Adagrad 3.54
categorical_hinge Adadelta 6.38
categorical_hinge Adam 5.46
categorical_hinge Nadam 3.83
logcosh SGD 14.08
logcosh RMSprop 7.88
logcosh Adagrad 2.62
logcosh Adadelta 12.83
logcosh Adam 4.67
logcosh Nadam 4.54'''

In [17]:
lst = []
for ln in rsl.split('\n'):
    elms = ln.split()
    elms[2] = float(elms[2])
    lst.append(elms)
lss_opt = pandas.DataFrame(lst, columns=['Loss', 'Optimizer', 'Score']).sort_values('Score')
lss_opt.to_csv('27 Losses and Optimizers.csv', encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)
lss_opt

,Loss,Optimizer,Score
44,logcosh,Adagrad,2.62
22,mean_squared_logarithmic_error,Adam,3.50
38,categorical_hinge,Adagrad,3.54
4,mean_squared_error,Adam,3.75
41,categorical_hinge,Nadam,3.83
47,logcosh,Nadam,4.54
46,logcosh,Adam,4.67
17,mean_absolute_percentage_error,Nadam,4.75
21,mean_squared_logarithmic_error,Adadelta,4.88
34,hinge,Adam,4.92


In [18]:
# for act1 in act_grid:
#     for act2 in act_grid:
#         try:
#             print(act1, act2, lstm_sp(10, 100, 100, [(100, act1), (24, act2)], 'logcosh', 'Adagrad', 10, 50))
#         except:
#             pass

In [19]:
rsl = '''softmax softmax 6.38
softmax elu 3.54
softmax selu 2.88
softmax softplus 11.12
softmax softsign 3.5
softmax relu 13.33
softmax tanh 4.08
softmax sigmoid 11.0
softmax hard_sigmoid 10.42
softmax linear 4.54
elu softmax 3.21
elu elu 2.54
elu selu 5.92
elu softplus 5.42
elu softsign 4.83
elu relu 17.58
elu tanh 5.0
elu sigmoid 5.88
elu hard_sigmoid 8.75
elu linear 4.12
selu softmax 4.92
selu elu 6.83
selu selu 4.67
selu softplus 5.38
selu softsign 5.96
selu relu 16.96
selu tanh 5.08
selu sigmoid 3.88
selu hard_sigmoid 7.5
selu linear 6.67
softplus softmax 3.88
softplus elu 2.54
softplus selu 7.88
softplus softplus 7.83
softplus softsign 5.67
softplus relu 17.67
softplus tanh 5.33
softplus sigmoid 6.33
softplus hard_sigmoid 6.04
softplus linear 4.46
softsign softmax 3.46
softsign elu 4.25
softsign selu 5.25
softsign softplus 6.58
softsign softsign 2.83
softsign relu 17.08
softsign tanh 2.71
softsign sigmoid 4.5
softsign hard_sigmoid 5.67
softsign linear 4.08
relu softmax 3.62
relu elu 3.38
relu selu 8.04
relu softplus 5.04
relu softsign 3.54
relu relu 7.88
relu tanh 4.25
relu sigmoid 4.08
relu hard_sigmoid 8.08
relu linear 5.29
tanh softmax 5.58
tanh elu 4.83
tanh selu 4.42
tanh softplus 4.08
tanh softsign 4.96
tanh relu 15.75
tanh tanh 2.33
tanh sigmoid 6.21
tanh hard_sigmoid 5.08
tanh linear 7.12
sigmoid softmax 3.29
sigmoid elu 5.38
sigmoid selu 6.12
sigmoid softplus 6.04
sigmoid softsign 2.33
sigmoid relu 16.29
sigmoid tanh 4.58
sigmoid sigmoid 6.58
sigmoid hard_sigmoid 6.5
sigmoid linear 2.83
hard_sigmoid softmax 4.17
hard_sigmoid elu 4.79
hard_sigmoid selu 6.96
hard_sigmoid softplus 5.62
hard_sigmoid softsign 5.08
hard_sigmoid relu 18.42
hard_sigmoid tanh 4.83
hard_sigmoid sigmoid 5.25
hard_sigmoid hard_sigmoid 4.21
hard_sigmoid linear 5.33
linear softmax 4.62
linear elu 3.67
linear selu 4.38
linear softplus 4.71
linear softsign 3.54
linear relu 18.5
linear tanh 3.04
linear sigmoid 2.67
linear hard_sigmoid 17.79
linear linear 4.71'''

In [20]:
lst = []
for ln in rsl.split('\n'):
    elms = ln.split()
    elms[2] = float(elms[2])
    lst.append(elms)
lss_opt = pandas.DataFrame(lst, columns=['Hidden', 'Output', 'Score']).sort_values('Score')
lss_opt.to_csv('27 Activations.csv', encoding='utf-8', index=False, quoting=csv.QUOTE_NONNUMERIC)
lss_opt

,Hidden,Output,Score
66,tanh,tanh,2.33
74,sigmoid,softsign,2.33
31,softplus,elu,2.54
11,elu,elu,2.54
97,linear,sigmoid,2.67
46,softsign,tanh,2.71
79,sigmoid,linear,2.83
44,softsign,softsign,2.83
2,softmax,selu,2.88
96,linear,tanh,3.04


In [18]:
achs = [[(10*i, 'tanh'), (24, 'tanh')] for i in range(2, 10)]

for ach in achs:
    try:
        print(lstm_sp(20, 100, 100, ach, 'logcosh', 'Adagrad', 5, 50))
    except:
        pass

3.96
7.83
5.54
5.83
5.88
7.21
5.96
3.58


In [15]:
achs = [[(10*i, 'tanh'), (24, 'tanh')] for i in range(2, 15)]

for ach in achs:
    try:
        print(lstm_sp(20, 100, 100, ach, 'logcosh', 'Adagrad', 5, 50))
    except:
        pass

3.12
7.08
5.12
5.5
4.71
5.92
5.04
2.96
7.75
4.67
5.21
6.83
6.5


In [12]:
achs = [[(10*i, 'tanh'), (24, 'tanh')] for i in range(2, 15)]

for ach in achs:
    try:
        print(lstm_sp(20, 100, 100, ach, 'logcosh', 'Adagrad', 5, 50))
    except:
        pass

4.46
6.58
4.33
5.88
4.79
6.29
4.88
3.54
7.12
4.38
5.12
6.62
7.29


In [11]:
achs = [[(i, 'tanh'), (24, 'tanh')] for i in range(10, 30)]

for ach in achs:
    try:
        print(ach, lstm_sp(20, 100, 100, ach, 'logcosh', 'Adagrad', 5, 50))
    except:
        pass

[(10, 'tanh'), (24, 'tanh')] 5.17
[(11, 'tanh'), (24, 'tanh')] 8.75
[(12, 'tanh'), (24, 'tanh')] 5.83
[(13, 'tanh'), (24, 'tanh')] 7.21
[(14, 'tanh'), (24, 'tanh')] 5.79
[(15, 'tanh'), (24, 'tanh')] 7.29
[(16, 'tanh'), (24, 'tanh')] 4.54
[(17, 'tanh'), (24, 'tanh')] 4.38
[(18, 'tanh'), (24, 'tanh')] 8.62
[(19, 'tanh'), (24, 'tanh')] 4.5
[(20, 'tanh'), (24, 'tanh')] 4.25
[(21, 'tanh'), (24, 'tanh')] 7.67
[(22, 'tanh'), (24, 'tanh')] 6.38
[(23, 'tanh'), (24, 'tanh')] 4.88
[(24, 'tanh'), (24, 'tanh')] 4.58
[(25, 'tanh'), (24, 'tanh')] 3.38
[(26, 'tanh'), (24, 'tanh')] 6.17
[(27, 'tanh'), (24, 'tanh')] 4.33
[(28, 'tanh'), (24, 'tanh')] 4.62
[(29, 'tanh'), (24, 'tanh')] 5.0


In [22]:
lstm_sp(20, 100, 100, [(25, 'tanh'), (24, 'tanh')], 'logcosh', 'Adagrad', 5, 50)

3.92

In [11]:
lstm_sp(20, 100, 100, [(25, 'tanh'), (24, 'tanh')], 'logcosh', 'Adagrad', 5, 50)

3.92

In [13]:
lstm_sp(20, 100, 100, [(25, 'tanh'), (24, 'tanh')], 'logcosh', 'Adagrad', 5, 50)

3.92

In [1]:
# Results now reproducible

In [27]:
for i in range(10, 150, 10):
    print(lstm_cv(3, 100, i, [(25, 'tanh'), (24, 'tanh')], 'logcosh', 'Adagrad', 5, 50))

4.04
4.25
9.0
4.46
6.75
2.33
4.96
4.29
6.54
5.62
5.25
3.08
4.75
5.92


In [11]:
for i in range(10, 150, 10):
    print(lstm_sp(20, i, 60, [(25, 'tanh'), (24, 'tanh')], 'logcosh', 'Adagrad', 5, 50))

4.38
5.17
7.92
4.88
4.0
11.04
6.17
4.25
6.5
5.67
3.96
5.46
5.67
3.54


In [12]:
for i in range(150, 250, 10):
    print(lstm_sp(20, i, 60, [(25, 'tanh'), (24, 'tanh')], 'logcosh', 'Adagrad', 5, 50))

5.0
5.17
8.0
4.88
4.04
11.04
6.08
4.25
6.42
5.29


In [15]:
lstm_sp(20, 100, 100, [(25, 'tanh'), (24, 'tanh')], 'logcosh', 'Adagrad', 5, 50)

6.33